In [1]:
import os
os.environ['HF_HOME'] = 'D:\\HuggingFace'
os.environ['TRANSFORMERS_CACHE'] = os.environ['HF_HOME']
os.environ['HUGGINGFACE_HUB_CACHE'] = os.environ['HF_HOME'] 

In [2]:
from transformers import pipeline

C:\Users\Турбо\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\utils\hub.py:123: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [4]:
def get_pred(query):
    answer = pipe(query, classes, multi_label=False)
    return [answer['labels'], answer['scores']]

In [5]:
def chunk_text(text, window_size=3, overlap=1):
    sentences = sent_tokenize(text)
    chunks = []
    for i in range(0, len(sentences), window_size-overlap):
        chunks.append(" ".join(sentences[i:i+window_size]))
    return chunks

In [6]:
def aggregate(document):
    # Делим документ
    chunks = chunk_text(document)

    # Предсказываем каждую часть
    preds = []
    for chun in chunks:
        preds.append(get_pred(chun))
        
    # Считаем взвешенную метрику
    scores = defaultdict(float)
    for pred in preds:
        for label, score in zip(pred[0], pred[1]):
            scores[label] += score

    return max(scores, key=scores.get)

In [7]:
import re

def process_info(file):
    with open(file, "rb") as f:
        reader = PyPDF2.PdfReader(f)
        text = ''.join([page.extract_text() for page in reader.pages])
    # Служебная информация
    text = re.sub(r'ISSN\s+\d{4}-\d{3,4}[^\n]*', '', text)
    text = re.sub(r'\d{4};\d{2}\(\d+\):\d+–\d+', '', text)

    # Авторы
    text = re.sub(r'[А-ЯЁ][а-яё]+\s+[А-ЯЁ][\.\s]+\s*[А-ЯЁ][\.\s]*', '', text)
    text = re.sub(r'[А-ЯЁ][а-яё]+\s+[А-ЯЁ][а-яё]+\s+[А-ЯЁ][\.\s]+\s*[А-ЯЁ][\.\s]*', '', text)
    text = re.sub(r'\d+[\s\w\.,–-]+(университет|институт|академия|центр)[^\n]*', '', text)

    # Сноски в квадратных скобках
    text = re.sub(r'\[\d+\]', '', text)  # [1], [2]
    text = re.sub(r'\[\d+[,-]\d+\]', '', text)  # [1-3], [4,5]
    text = re.sub(r'\[[A-Za-z]+\d*\]', '', text)  # [A1], [B]
    
    # email
    text = re.sub(r'\S+@\S+', '', text)
    
    # английские разделы 
    text = re.sub(r'Abstract[^\n]*[\s\S]*?(?=\n[А-ЯЁ]|$)', '', text)
    text = re.sub(r'Keywords[^\n]*[\s\S]*?(?=\n[А-ЯЁ]|$)', '', text)
    text = re.sub(r'For citation[^\n]*[\s\S]*?(?=\n[А-ЯЁ]|$)', '', text)
    
    # ссылки
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)
    text = re.sub(r'DOI:\s*\S+', '', text)
    
    # библиография
    text = re.sub(r'Список\s+источников[\s\S]*', '', text)
    text = re.sub(r'References[\s\S]*', '', text)
    
    # спец.символы
    text = text.replace('\xa0', ' ').replace('•', '')
    text = re.sub(r'-\s+', '', text)  
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'•\s*\n', '', text)
    
    # оставшиеся английские фрагменты
    text = re.sub(r'(?:[A-Za-z-]+\s){3,}[A-Za-z-]*', '', text)
    
    return text

In [11]:
from transformers import pipeline
import psycopg2
from psycopg2 import sql, errors
from collections import defaultdict
from nltk import sent_tokenize
import PyPDF2
import pathlib

DB_CONFIG = {
    "dbname": "Electronic_library",
    "user": "postgres",
    "password": "Turbo",
    "host": "localhost",
    "port": "5432"
}

pipe = pipeline(
    "zero-shot-classification", 
    model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli", 
    hypothesis_template="Определи тему научной статьи '{}'", 
    disable_hf_device_logging=True,
)

classes = ['Education', 'Art', 'Sport', 'Chemical', 'Space']

def create_connection():
    """Устанавливает соединение с PostgreSQL."""
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        print("✅ Подключение к PostgreSQL успешно")
        return conn
    except Exception as e:
        print(f"❌ Ошибка подключения: {e}")
        return None


def create_table(conn):
    """Создает таблицу для хранения классифицированных статей."""
    create_table_query = """
    CREATE TABLE IF NOT EXISTS articles (
        id SERIAL PRIMARY KEY,
        url TEXT NOT NULL UNIQUE,
        education BOOLEAN DEFAULT FALSE,
        art BOOLEAN DEFAULT FALSE,
        sport BOOLEAN DEFAULT FALSE,
        chemical BOOLEAN DEFAULT FALSE,
        space BOOLEAN DEFAULT FALSE
    );
    """
    try:
        with conn.cursor() as cursor:
            cursor.execute(create_table_query)
            conn.commit()
        print("✅ Таблица 'articles' создана или уже существует")
    except Exception as e:
        print(f"❌ Ошибка создания таблицы: {e}")
        conn.rollback()


def predict_class(text):
    """Определяет класс текста."""
    try:
        result = aggregate(text)
        return result
    except Exception as e:
        print(f"❌ Ошибка классификации: {e}")
        return None


def insert_article(conn, url, text):
    """Добавляет статью в БД, устанавливая True в нужный столбец, остальные NULL."""
    predicted_class = predict_class(text)
    if not predicted_class:
        return False

    try:
        with conn.cursor() as cursor:
            # Создаем словарь для обновления (все NULL, кроме predicted_class)
            data = {class_name.lower(): (True if class_name == predicted_class else None) 
                   for class_name in classes}
            data['url'] = url

            # Формируем SQL-запрос
            columns = sql.SQL(', ').join(map(sql.Identifier, data.keys()))
            values = sql.SQL(', ').join([sql.Placeholder()] * len(data))
            
            query = sql.SQL("""
            INSERT INTO articles ({columns})
            VALUES ({values})
            ON CONFLICT (url) DO UPDATE SET
                {update_clause}
            """).format(
                columns=columns,
                values=values,
                update_clause=sql.SQL(', ').join([
                    sql.SQL("{} = EXCLUDED.{}").format(
                        sql.Identifier(k),
                        sql.Identifier(k)
                    ) for k in data.keys() if k != 'url'
                ])
            )
            
            cursor.execute(query, list(data.values()))
            conn.commit()

        print(f"✅ Статья '{url}' добавлена в класс '{predicted_class}'")
        return True
    except Exception as e:
        print(f"❌ Ошибка вставки: {e}")
        conn.rollback()
        return False


if __name__ == "__main__":
    conn = create_connection()
    if conn:
        create_table(conn)

        insert_article(
            conn,
            url="https://elibrary.ru/download/elibrary_26738028_39746529.pdf",
            text=process_info("art.pdf")
        )

        conn.close()

✅ Подключение к PostgreSQL успешно
✅ Таблица 'articles' создана или уже существует
✅ Статья 'https://elibrary.ru/download/elibrary_26738028_39746529.pdf' добавлена в класс 'Art'
